**POZNÁMKA: Tento notebook je určený pre platformu Google Colab. Je však možné ho spustiť (možno s drobnými úpravami) aj ako štandardný Jupyter notebook.** 



In [ ]:
#@title -- Installation of Packages -- { display-mode: "form" }
import sys
!{sys.executable} -m pip install datasets
!{sys.executable} -m pip install umap-learn
!{sys.executable} -m pip install git+https://github.com/michalgregor/class_utils.git

In [ ]:
#@title -- Import of Necessary Packages -- { display-mode: "form" }
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import make_pipeline

from datasets import load_dataset
from class_utils import imscatter
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import numpy as np
import umap

## Znižovanie rozmeru dát

Cieľom metód znižovania rozmeru dát je znížiť rozmer dát tak, aby sa pritom stratilo čo najmenej užitočnej informácie. Znižovanie rozmeru môže mať rôzne ciele, napríklad:

* chceme znížiť výpočtové nároky na spracovanie dát;
* vizualizácia vysokorozmerných dát;
* ...
My si teraz budeme ilustrovať postup znižovania rozmeru dát na účel vizualizácie.

### Načítanie dát

Na ilustráciu použijeme dátovú množinu [Fashion MNIST](https://huggingface.co/datasets/fashion_mnist), ktorá obsahuje obrázky rôznych typov obuvi a oblečenia v malom rozlíšení $28 \times 28$ pixelov.

Obrázky v dátovej množine sú roztriedené do nasledujúcich tried:

label id | label       |  | label id | label     
-------- | ----------- | - | -------- | ----------
**0**    | T-shirt/top |  | **5**    | Sandal    
**1**    | Trouser     |  | **6**    | Shirt     
**2**    | Pullover    |  | **7**    | Sneaker   
**3**    | Dress       |  | **8**    | Bag       
**4**    | Coat        |  | **9**    | Ankle boot
Dátovú množinu načítame veľmi jednoducho, pretože balíček `datasets` od firmy HuggingFace obsahuje pribalenú funkciu, ktorá to umožňuje. Stačí len zavolať funkciu `load_dataset` a ako dátovú množinu špecifikovať `"fashion_mnist"`. S predvolenými parametrami by sme takto získali dátovú množinu rozdelenú na dve časti: na trénovaciu a testovaciu časť. Keďže plánujeme realizovať len vizualizáciu a nie kontrolované učenie, nebudeme testovaciu množinu potrebovať. Špecifikujeme preto `split='train+test'` – takto naša dátová množina nebude rozdelená na dve časti.



In [ ]:
dataset = load_dataset("fashion_mnist", split='train+test')
dataset

Jedna vcelku užitočná vlastnosť dátových množín načítaných pomocou `load_dataset` je, že majú atribút `.info`, ktorý obsahuje metadáta o dátovej množine. Je takto možné si napríklad zobraziť krátky opis dátovej množiny alebo získať názvy tried:



In [ ]:
print(dataset.info.description)

In [ ]:
class_names = dataset.info.features['label'].names
print(class_names)

In [ ]:
X = np.asarray([np.asarray(img) for img in dataset['image']])
Y = np.asarray(dataset['label'])

Aby sme mali predstavu, ako dáta vyzerajú, zobrazíme si ďalej niekoľko náhodne zvolených vzoriek:



In [ ]:
fig, axes = plt.subplots(5, 5)
fig.set_size_inches([7, 7])

for ax_row in axes:
    for ax in ax_row:
        ind = np.random.randint(0, X.shape[0])
        ax.imshow(X[ind], cmap='Greys')
        ax.set_title(class_names[Y[ind]])
        ax.axis('off')
    
plt.subplots_adjust(hspace=0.5)

### Predspracovanie

Pripomíname, že bežne by sme dáta pred aplikáciou PCA či UMAP **štandardizovali**  (preškálovali každý stĺpec tak, aby jeho priemer bol nula a smerodajná odchýlka bola 1). Robí sa to preto, aby metóda nepovažovala určité stĺpce za dôležitejšie len preto, že hodnoty majú väčšiu škálu. V tomto prípade sú však naše dáta obrázové, takže každý rozmer (každý pixel) už má rovnakú mierku.



In [ ]:
# WE DO NOT NEED THIS BECAUSE WE HAVE AN IMAGE DATASET

# input_preproc = make_pipeline(
#     SimpleImputer(),
#     StandardScaler()
# )

# X_preproc = input_preproc.fit_transform(X.reshape(X.shape[0], -1))
# X_preproc = X_preproc.reshape(X.shape)
# X = X_preproc

### Znižovanie rozmeru dát pomocou PCA a vizualizácia

Keďže obrázky majú rozmer $28 \times 28$ pixelov, priestor je 784-rozmerný. Ak chceme vizualizovať jeho štruktúru, musíme dáta redukovať do 2-rozmerného priestoru. Tým prirodzene veľké množstvo informácie stratíme, ale v dobrom prípade sa budeme stále schopní dozvedieť veľa o štruktúre priestoru.

Ako prvú metódu na znižovanie rozmeru otestujeme metódu PCA. Ide o metódu, ktorá je veľmi rýchla, ale vie využiť len lineárne závislosti v dátach – nie nelineárne. Pri niektorých dátových množinách to však stačí.



In [ ]:
pca = PCA()
points_pca = pca.fit_transform(X.reshape((X.shape[0], -1)))

Body pred vizualizáciou premiešame – v pôvodnej dátovej množine sú zotriedené podľa tried. Ak chceme vidieť, či je PCA schopná od seba jednotlivé triedy oddeliť, premiešanie je potrebné: inak by body z neskoršej triedy mohli úplne prekryť body z niektorej skoršej triedy, čo by viedlo ku falošnému dojmu, že triedy sú dobre oddelené.



In [ ]:
perm_ind = np.random.permutation(points_pca.shape[0])
xx = points_pca[perm_ind]
yy = Y[perm_ind]

Nakoniec nám zostáva už len vizualizovať všetky body, zafarbené podľa triedy. Ako vidno, metóda PCA nie je schopná dobre oddeliť jednotlivé triedy. Hoci niektoré triedy sú pomerne jasne separované (napr. bag a trouser), celkovo obrázok nie je čitateľný.



In [ ]:
plt.figure(figsize=[10, 7])
plt.scatter(xx[:, 0], xx[:, 1], c=yy,
            cmap=plt.cm.get_cmap('jet', len(class_names)),
            rasterized=True)
cbar = plt.colorbar()
cbar.set_ticks(range(len(class_names)))
cbar.set_ticklabels(class_names)
plt.xlabel("dim 1")
plt.ylabel("dim 2")

Ešte menej by sme videli, keby sme body nezafarbili:



In [ ]:
plt.figure(figsize=[10, 7])
plt.scatter(xx[:, 0], xx[:, 1], rasterized=True)
plt.xlabel("dim 1")
plt.ylabel("dim 2")

#### Poznámka: Rasterizácia časti obrázka

Všimnite si, že pri vykresľovaní bodov používame argument `rasterized=True`. Ten indikuje, že príslušná časť grafu sa má rasterizovať. Pri zobrazovaní veľmi veľkých počtov bodov je to výhodné urobiť – inak by obrázok po uložení do vektorového formátu bolo veľmi ťažké zobraziť. 

Obrázok sa prirodzene dá uložiť do rastrového formátu (ako jpeg alebo png) aj ako celok — ibaže potom sú v rastrovom formáte aj osi a ďalšie časti obrázka. Tomu sa je vo všeobecnosti lepšie vyhnúť: hlavne v prípade, že sa má obrázok použiť v nejakom texte: napríklad v záverečnej práci, v článku a pod.

V prípade, kedy je vektorový obrázok príliš zložitý, rasterizácia len jednej, problematickej časti predstavuje dobrý kompromis.



### Znižovanie rozmeru dát pomocou UMAP a vizualizácia

Znižovanie rozmeru dát pomocou metódy UMAP bude trvať podstatne dlhšie než pomocou metódy PCA. Na druhej strane sa dá očakávať, že aj výsledky budú podstatne lepšie, pretože metóda UMAP nie je obmedzená len na lineárne zákonitosti v dátach.

V kóde, ktorý sme použili vyššie, stačí doslova prepísať "PCA" na "UMAP", pretože obe metódy implementujú unifikované rozhranie podľa balíčka `scikit-learn`. If chceme vidieť trochu viac informácií o tom, ako UMAP postupuje, môžeme pridať argument `verbose=True`.



In [ ]:
um = umap.UMAP(verbose=True)
points_umap = um.fit_transform(X.reshape((X.shape[0], -1)))

In [ ]:
perm_ind = np.random.permutation(points_umap.shape[0])
xx = points_umap[perm_ind]
yy = Y[perm_ind]
xt = X[perm_ind]

In [ ]:
plt.figure(figsize=[10, 7])
cmap = plt.cm.get_cmap('jet', len(class_names))
plt.scatter(xx[:, 0], xx[:, 1], c=yy,
            cmap=cmap,
            rasterized=True)
cbar = plt.colorbar()
cbar.set_ticks(range(len(class_names)))
cbar.set_ticklabels(class_names)
plt.xlabel("dim 1")
plt.ylabel("dim 2")

Z tohto obrázka sa už o štruktúre dátovej množiny dozvedáme omnoho viac. Vidno, že vzorky sú rozdelené do 4 veľkých skupín. Jedna obsahuje nohavice, druhá kabelky, tretia zmiešava rôzne typy topánok a štvrtá rôzne typy tričiek, šiat a kabátov.

Vidíme tiež, že zatiaľ čo tričká a kabáty sú dosť premiešané, topánky sú aj vo vnútri spoločného zhluku pomerne dobre oddeliteľné.

### Pokročilejšia vizualizácia

Z vizualizácie pomocou metódy UMAP vidíme, že z nejakého dôvodu existuje spojitý prechod medzi tričkami a kabelkami. Bolo by zaujímavé zistiť, aké vzorky sú na rozhraní oboch zhlukov. Aby sme to zistili, môžeme si do grafu namiesto všetkých bodov vykrelisť len časť z nich, ale vizualizovať na ich pozíciách aj pôvodné obrázky. To nám poskytne plnšiu vizuálnu informáciu o charaktere zhlukov.

Použijeme pomocnú funkciu s podobným rozhraním ako `scatter`, ktorá však bude namiesto bodov vykresľovať obrázky.



In [ ]:
num2show = 800

plt.figure(figsize=[15, 10])
imscatter(xx[:num2show, 0], xx[:num2show, 1],
          xt[:num2show], cmap='Greys', zoom=1.2,
          frame_c=yy[:num2show], frame_cmap=cmap,
          frame_linewidth=2)
plt.xlabel("dim 1")
plt.ylabel("dim 2")

Z obrázka by malo byť vidno, že kabelky, ktoré susedia s obrázkami tričiek a šiat naozaj menia postupne tvar, takže niektoré môžu byť v nízkom rozlíšení a čiernobielych farbách zameniteľné s oblečením.

